In [1]:
import keras_tuner
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import pandas as pd

2024-04-28 08:11:38.258592: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 08:11:38.271972: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 08:11:38.388016: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 08:11:39.686648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_df = pd.read_csv("adult/data/shadow_model_outputs_proba-new.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [4]:
adv_df_test = pd.read_csv("adult/data/shadow_model_outputs_proba-new_test_set.csv")
#adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [13]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(keras.layers.Dense(hp.Int('units_dense0', min_value=24, max_value=40, step=4)))
        #if hp.Boolean("relu_dense0"):
        #    model.add(keras.layers.Activation(keras.activations.relu))
        #model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.01, max_value=0.03, step=0.01)))
        model.add(keras.layers.Dense(hp.Int('units_dense1', min_value=6, max_value=14, step=2)))
        #if hp.Boolean("relu_dense1"):
        #    model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dense(1))
        model.compile(optimizer="adam", loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
        return model
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [10, 18, 24]),
            **kwargs,
        )


In [14]:
from keras_tuner import Objective
tuner = RandomSearch(
    MyHyperModel(),
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = Objective(name="val_r2_score", direction="max"),
    max_trials = 50,
    executions_per_trial = 4,
    tuner_id="17.01.regression.adversary.cgpt"
)

In [ ]:
tuner.search(adv_X, adv_y, epochs=200, validation_data=(adv_X_test, adv_y_test), callbacks=[keras.callbacks.EarlyStopping('val_r2_score', patience=30)])

Trial 36 Complete [00h 02m 30s]
val_r2_score: 0.3863646537065506

Best val_r2_score So Far: 0.38754263520240784
Total elapsed time: 01h 40m 05s

Search: Running Trial #37

Value             |Best Value So Far |Hyperparameter
24                |28                |units_dense0
14                |6                 |units_dense1
18                |10                |batch_size
Epoch 1/200
200/200 [==============================] - 2s 6ms/step - loss: 46.9086 - r2_score: -702.6303 - val_loss: 0.1252 - val_r2_score: -0.8780
Epoch 2/200
200/200 [==============================] - 1s 4ms/step - loss: 0.0047 - r2_score: 0.9292 - val_loss: 0.1079 - val_r2_score: -0.6191
Epoch 3/200
200/200 [==============================] - 1s 4ms/step - loss: 0.0024 - r2_score: 0.9647 - val_loss: 0.1082 - val_r2_score: -0.6226
Epoch 4/200
200/200 [==============================] - 1s 4ms/step - loss: 0.0016 - r2_score: 0.9758 - val_loss: 0.1070 - val_r2_score: -0.6051
Epoch 5/200
200/200 [=======================

In [ ]:
models = tuner.get_best_models(num_models=5)

In [17]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_r2_score", direction="max")

Trial 07 summary
Hyperparameters:
units_dense0: 28
units_dense1: 6
batch_size: 10
Score: 0.38754263520240784

Trial 35 summary
Hyperparameters:
units_dense0: 40
units_dense1: 6
batch_size: 24
Score: 0.3863646537065506

Trial 32 summary
Hyperparameters:
units_dense0: 36
units_dense1: 6
batch_size: 10
Score: 0.3810035139322281

Trial 29 summary
Hyperparameters:
units_dense0: 24
units_dense1: 8
batch_size: 24
Score: 0.3312181830406189

Trial 05 summary
Hyperparameters:
units_dense0: 32
units_dense1: 12
batch_size: 10
Score: 0.32719068229198456

Trial 27 summary
Hyperparameters:
units_dense0: 24
units_dense1: 14
batch_size: 10
Score: 0.3196559399366379

Trial 17 summary
Hyperparameters:
units_dense0: 40
units_dense1: 12
batch_size: 24
Score: 0.302568644285202

Trial 23 summary
Hyperparameters:
units_dense0: 32
units_dense1: 6
batch_size: 10
Score: 0.29553239047527313

Trial

In [ ]:
best_model = models[0]

In [10]:
best_model.build(input_shape=(None,adv_X.shape[1],))

In [12]:
best_model.save("models/adv_new_0.40_test_r2")
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)

INFO:tensorflow:Assets written to: adv_new_0.40_test_r2/assets


INFO:tensorflow:Assets written to: adv_new_0.40_test_r2/assets


In [40]:
best_model.predict(adv_X_test)

57/57 [==============================] - 0s 2ms/step


array([[0.5596606 ],
       [0.4603508 ],
       [0.4807027 ],
       ...,
       [0.62426335],
       [0.4381116 ],
       [0.584784  ]], dtype=float32)

## Manual Tuning (new)

In [23]:
manual_adversary.load_weights("adult/models/adult_adv_0.72_test_r2.keras")
manual_adversary.evaluate(adv_X_test, adv_y_test)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0186 - r2_score: 0.7307  


[0.018278300762176514, 0.7281985878944397]

In [22]:
checkpoint_filepath = 'models/manual_tuning_checkpoints/keras.weights.h5'


currentmax = 0.6
for i in range(100):
    manual_adversary = keras.Sequential()
    manual_adversary.add(keras.Input(shape=(adv_X.shape[1],)))
    manual_adversary.add(keras.layers.Dense(28))
    manual_adversary.add(keras.layers.Dense(6))
    manual_adversary.add(keras.layers.Dense(1))
    manual_adversary.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.R2Score()])
    #manual_adversary.load_weights("adv_new_new_keras_xx_test_r2.keras")
    history = manual_adversary.fit(
        adv_X, 
        adv_y, 
        epochs=200, 
        validation_data=(adv_X_test, adv_y_test), 
        verbose=0,
        batch_size=10,
        callbacks=[
            keras.callbacks.EarlyStopping('val_r2_score', mode='max', patience=20, verbose=1),
            keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_r2_score',
                mode='max',
                save_best_only=True)
        ])
    newmax = max(history.history['val_r2_score'])
    if newmax > currentmax:
        print(f"new max r2: {newmax}")
        currentmax = newmax
        manual_adversary.load_weights("models/manual_tuning_checkpoints/keras.weights.h5")
        manual_adversary.save("adv_new_new_keras_xx_test_r2.keras")

Epoch 103: early stopping
Epoch 21: early stopping
Epoch 84: early stopping
Epoch 74: early stopping
Epoch 24: early stopping
Epoch 22: early stopping
Epoch 102: early stopping
Epoch 37: early stopping
Epoch 39: early stopping
Epoch 32: early stopping
Epoch 21: early stopping
new max r2: 0.7281985878944397
Epoch 38: early stopping


KeyboardInterrupt: 

0.27153974771499634

In [62]:
manual_adversary.load_weights("models/manual_tuning_checkpoints/keras.weights.h")

In [63]:
manual_adversary.evaluate(adv_X_test, adv_y_test)

57/57 [==============================] - 0s 2ms/step - loss: 0.0232 - r2_score: 0.6518


[0.02321377582848072, 0.6517935395240784]

In [64]:
manual_adversary.save("adv_new_0.65_test_r2")

INFO:tensorflow:Assets written to: adv_new_0.65_test_r2/assets


INFO:tensorflow:Assets written to: adv_new_0.65_test_r2/assets
